# Here the Task Advanced Grid Function #18 will be developed

## ToDo:
- Function grid
- input(sensorid, trafficIndex, shape=Diameter)
- Output(Grid)
- A Function that can be used to create a Grid with the options of different diameter.
- Grid check, if empty squares, and fill them with the mean from value around

## Accteptance:
- In the Dashboard dont show empty squares.
- It is possible to select different square sizes.


In [1]:
import pandas as pd
import numpy as np
import branca.colormap as cm  # Used for color gradient
import folium
import geopandas
import joblib
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
# Get Detectors

df_sensors = pd.read_csv(r"C:\Users\rueed\OneDrive\HSLU\3 Semester\DSPRO 1\HSLU_DSPRO1_TrafficStatus\data\RawDataLondon\London_detectors.csv")
df_sensors.head()

,detid,length,pos,fclass,road,limit,citycode,lanes,linkid,long,lat
0,EAST_N04/161x1,0.303585,0.261157,secondary,Homerton Road,NaN,london,1.0,5082.0,-0.021497,51.550929
1,EAST_N04/161y1,0.103679,0.063417,primary,Eastway,NaN,london,1.0,5091.0,-0.020899,51.550704
2,EAST_N04/162a1,0.260623,0.117906,secondary,Homerton Road,NaN,london,1.0,5083.0,-0.022649,51.550907
3,EAST_N04/162a2,0.216874,0.117942,secondary,Homerton Road,NaN,london,1.0,5084.0,-0.022617,51.550880
4,EAST_N04/163f1,0.344754,0.329789,primary,Eastway,NaN,london,1.0,5092.0,-0.019288,51.552281


In [3]:
# Get Sensors from Models
 
def get_knn_prediction(models_path, weekday, interval_values=[
               0, 3600, 7200, 10800, 14400, 18000, 21600, 25200, 28800, 32400, 
               36000, 39600, 43200, 46800, 50400, 54000, 57600, 61200, 64800, 
               68400, 72000, 75600, 79200, 82800]):
    
    X_values = pd.DataFrame(interval_values, columns=['interval'])
    X_values['weekday'] = weekday
    
    predictions = []
    
    for model_filename in os.listdir(models_path):
        model_path = os.path.join(models_path, model_filename)
        if os.path.isfile(model_path):
            # Load the KNN model
            sensor_model = joblib.load(model_path)
            y_pred = sensor_model.predict(X_values)
            
            # Store predictions in DataFrame format
            predictions.append(pd.DataFrame({
                'traffic': y_pred,
                'detid': model_filename.replace('-', '/').replace('.pkl', ''),
                'interval': X_values['interval'],
            }))
        
    return pd.concat(predictions)


In [4]:
df_monday = get_knn_prediction(r"C:\Users\rueed\OneDrive\HSLU\3 Semester\DSPRO 1\data\knn04", 0)
df_monday.head()

,traffic,detid,interval
0,53.5,CNTR_N00/005g1,0
1,37.6,CNTR_N00/005g1,3600
2,20.3,CNTR_N00/005g1,7200
3,12.2,CNTR_N00/005g1,10800
4,19.0,CNTR_N00/005g1,14400


In [5]:
# Fullfill the Sensors Infos
df_real = pd.merge(df_monday, df_sensors, on='detid', how='left')


df_real = df_real[df_real['interval'] == 0]
df_real.head()

,traffic,detid,interval,length,pos,fclass,road,limit,citycode,lanes,linkid,long,lat
0,53.500000,CNTR_N00/005g1,0,0.237532,0.224619,tertiary,Temple Place,NaN,london,1.0,463.0,-0.111488,51.511081
24,51.700000,CNTR_N00/005g2,0,0.238617,0.224596,tertiary,Temple Place,NaN,london,1.0,464.0,-0.111477,51.511047
48,17.000000,CNTR_N00/005x1,0,0.420449,0.335635,tertiary,Temple Avenue,NaN,london,1.0,3765.0,-0.107033,51.511085
72,12.500000,CNTR_N00/005x2,0,0.419348,0.335239,tertiary,Temple Avenue,NaN,london,1.0,3766.0,-0.107035,51.511054
96,22.333333,CNTR_N01/001b1,0,0.046710,0.019461,tertiary,Dover Street,NaN,london,1.0,5171.0,-0.140822,51.507659


### Develope for the function to fillout the Grid

In [6]:
def grid(df, sensorid_col, trafficIndex_col, shape=0.01):
    """
    Input:
    - df: DataFrame containing sensor data with longitude and latitude
    - sensorid_col: column name for sensor ids
    - trafficIndex_col: column name for traffic indices (e.g. length or traffic volume)
    - shape: the size of the grid (diameter of the cell)
    
    Output:
    - A DataFrame with the grid and the mean trafficIndex for each grid cell.
    """
    # 1. Round the coordinates to a precision based on 'shape' (grid diameter)
    df['long_rounded'] = (df['long'] // shape) * shape
    df['lat_rounded'] = (df['lat'] // shape) * shape
    
    # 2. Create a grid ID based on the rounded coordinates
    df['grid_id'] = df['long_rounded'].astype(str) + "_" + df['lat_rounded'].astype(str)
    
    # 3. Calculate the mean of the trafficIndex for each grid and count sensors
    grid = df.groupby('grid_id').agg(
        mean_trafficIndex=(trafficIndex_col, 'mean'),
        sensors_in_grid=(sensorid_col, 'count'),
        long_rounded=('long_rounded', 'first'),
        lat_rounded=('lat_rounded', 'first')
    ).reset_index()

    return grid

In [7]:
grid_data = grid(df_real, sensorid_col='detid', trafficIndex_col='traffic', shape=0.01)

# View the grid data
grid_data

,grid_id,mean_trafficIndex,sensors_in_grid,long_rounded,lat_rounded
0,-0.01_51.47,8.416667,6,-0.01,51.47
1,-0.01_51.480000000000004,7.153846,13,-0.01,51.48
2,-0.01_51.5,5.727750,29,-0.01,51.50
3,-0.01_51.51,9.723117,55,-0.01,51.51
4,-0.01_51.53,17.077778,12,-0.01,51.53
...,...,...,...,...,...
223,0.0_51.51,4.640890,46,0.00,51.51
224,0.0_51.52,1.666667,3,0.00,51.52
225,0.0_51.53,3.182313,21,0.00,51.53
226,0.0_51.54,5.167560,16,0.00,51.54


In [8]:
grid_data.get('grid_id').to_list()

['-0.01_51.47',
 '-0.01_51.480000000000004',
 '-0.01_51.5',
 '-0.01_51.51',
 '-0.01_51.53',
 '-0.01_51.54',
 '-0.01_51.550000000000004',
 '-0.02_51.47',
 '-0.02_51.480000000000004',
 '-0.02_51.5',
 '-0.02_51.51',
 '-0.02_51.52',
 '-0.02_51.53',
 '-0.02_51.54',
 '-0.02_51.550000000000004',
 '-0.03_51.46',
 '-0.03_51.47',
 '-0.03_51.480000000000004',
 '-0.03_51.5',
 '-0.03_51.51',
 '-0.03_51.52',
 '-0.03_51.53',
 '-0.03_51.54',
 '-0.03_51.550000000000004',
 '-0.04_51.47',
 '-0.04_51.51',
 '-0.04_51.52',
 '-0.04_51.53',
 '-0.04_51.54',
 '-0.04_51.550000000000004',
 '-0.05_51.47',
 '-0.05_51.49',
 '-0.05_51.5',
 '-0.05_51.51',
 '-0.05_51.52',
 '-0.05_51.53',
 '-0.05_51.54',
 '-0.06_51.47',
 '-0.06_51.480000000000004',
 '-0.06_51.49',
 '-0.06_51.5',
 '-0.06_51.51',
 '-0.06_51.52',
 '-0.06_51.53',
 '-0.06_51.54',
 '-0.06_51.550000000000004',
 '-0.07_51.46',
 '-0.07_51.47',
 '-0.07_51.480000000000004',
 '-0.07_51.49',
 '-0.07_51.5',
 '-0.07_51.51',
 '-0.07_51.52',
 '-0.07_51.53',
 '-0.07_51.5

In [9]:
grid_data.get('lat_rounded').unique()

array([51.47, 51.48, 51.5 , 51.51, 51.53, 51.54, 51.55, 51.52, 51.46,
       51.49])

In [10]:
grid_data.get('long_rounded').unique()

array([-0.01, -0.02, -0.03, -0.04, -0.05, -0.06, -0.07, -0.08, -0.09,
       -0.11, -0.12, -0.13, -0.14, -0.15, -0.16, -0.17, -0.18, -0.19,
       -0.1 , -0.21, -0.22, -0.2 ,  0.01,  0.02,  0.03,  0.  ])

When grid is full with the used resolution, there should be 260 rows.
At the moment there are 228 because 32 fields have 0 sensors. 

Zuerst wird ein Grid erstellt für alle Rectangles:

In [ ]:
def create_complete_grid(df, shape=0.01):
    """
    Creates a complete grid based on the minimum and maximum longitude and latitude in the input DataFrame.
    Input:
    - df: DataFrame containing sensor data with longitude and latitude
    - shape: the size of the grid (diameter of the cell)
    
    Output:
    - A DataFrame with the complete grid and the mean trafficIndex for each grid cell.
    """
    
    decimal_places = abs(int(round(-np.log10(shape), 0)))
    
    df['long_rounded'] = df['long_rounded'].round(decimal_places)
    df['lat_rounded'] = df['lat_rounded'].round(decimal_places)
    
    # min max values
    min_long = df['long_rounded'].min()
    max_long = df['long_rounded'].max()
    min_lat = df['lat_rounded'].min()
    max_lat = df['lat_rounded'].max()
    
    # Grid from min to max in steps of shape 
    np.set_printoptions(suppress=True) 
    all_longs = np.arange(min_long, max_long + shape, shape)
    all_lats = np.arange(min_lat, max_lat, shape)
    
    print(all_longs.shape)
    print(all_lats.shape)
    
    # Create a complete grid
    complete_grid = pd.MultiIndex.from_product([all_longs, all_lats], names=['long_rounded', 'lat_rounded']).to_frame(index=False)
    print(complete_grid)
    
    # Create id for each grid cell
    complete_grid['grid_id'] = complete_grid.apply(
        lambda row: f"{round(row['long_rounded'], decimal_places)}_{round(row['lat_rounded'], decimal_places)}", axis=1
    )
    print(complete_grid)
    
    # set mean_trafficIndex to None
    complete_grid['mean_trafficIndex'] = None
    
    return complete_grid

In [32]:
df_test = create_complete_grid(df_real, shape=0.01)
df_test

(26,)
(9,)
     long_rounded  lat_rounded
0           -0.22        51.46
1           -0.22        51.47
2           -0.22        51.48
3           -0.22        51.49
4           -0.22        51.50
..            ...          ...
229          0.03        51.50
230          0.03        51.51
231          0.03        51.52
232          0.03        51.53
233          0.03        51.54

[234 rows x 2 columns]
     long_rounded  lat_rounded      grid_id
0           -0.22        51.46  -0.22_51.46
1           -0.22        51.47  -0.22_51.47
2           -0.22        51.48  -0.22_51.48
3           -0.22        51.49  -0.22_51.49
4           -0.22        51.50   -0.22_51.5
..            ...          ...          ...
229          0.03        51.50    0.03_51.5
230          0.03        51.51   0.03_51.51
231          0.03        51.52   0.03_51.52
232          0.03        51.53   0.03_51.53
233          0.03        51.54   0.03_51.54

[234 rows x 3 columns]


,long_rounded,lat_rounded,grid_id,mean_trafficIndex
0,-0.22,51.46,-0.22_51.46,None
1,-0.22,51.47,-0.22_51.47,None
2,-0.22,51.48,-0.22_51.48,None
3,-0.22,51.49,-0.22_51.49,None
4,-0.22,51.50,-0.22_51.5,None
...,...,...,...,...
229,0.03,51.50,0.03_51.5,None
230,0.03,51.51,0.03_51.51,None
231,0.03,51.52,0.03_51.52,None
232,0.03,51.53,0.03_51.53,None


In [13]:
df_test.get('lat_rounded').unique()

array([51.46, 51.47, 51.48, 51.49, 51.5 , 51.51, 51.52, 51.53, 51.54,
       51.55])

In [14]:
df_test.get('long_rounded').unique()

array([-0.22, -0.21, -0.2 , -0.19, -0.18, -0.17, -0.16, -0.15, -0.14,
       -0.13, -0.12, -0.11, -0.1 , -0.09, -0.08, -0.07, -0.06, -0.05,
       -0.04, -0.03, -0.02, -0.01,  0.  ,  0.01,  0.02,  0.03])

In [15]:
df_test.get('grid_id').to_list()

['-0.22_51.46',
 '-0.22_51.47',
 '-0.22_51.48',
 '-0.22_51.49',
 '-0.22_51.5',
 '-0.22_51.51',
 '-0.22_51.52',
 '-0.22_51.53',
 '-0.22_51.54',
 '-0.22_51.55',
 '-0.21_51.46',
 '-0.21_51.47',
 '-0.21_51.48',
 '-0.21_51.49',
 '-0.21_51.5',
 '-0.21_51.51',
 '-0.21_51.52',
 '-0.21_51.53',
 '-0.21_51.54',
 '-0.21_51.55',
 '-0.2_51.46',
 '-0.2_51.47',
 '-0.2_51.48',
 '-0.2_51.49',
 '-0.2_51.5',
 '-0.2_51.51',
 '-0.2_51.52',
 '-0.2_51.53',
 '-0.2_51.54',
 '-0.2_51.55',
 '-0.19_51.46',
 '-0.19_51.47',
 '-0.19_51.48',
 '-0.19_51.49',
 '-0.19_51.5',
 '-0.19_51.51',
 '-0.19_51.52',
 '-0.19_51.53',
 '-0.19_51.54',
 '-0.19_51.55',
 '-0.18_51.46',
 '-0.18_51.47',
 '-0.18_51.48',
 '-0.18_51.49',
 '-0.18_51.5',
 '-0.18_51.51',
 '-0.18_51.52',
 '-0.18_51.53',
 '-0.18_51.54',
 '-0.18_51.55',
 '-0.17_51.46',
 '-0.17_51.47',
 '-0.17_51.48',
 '-0.17_51.49',
 '-0.17_51.5',
 '-0.17_51.51',
 '-0.17_51.52',
 '-0.17_51.53',
 '-0.17_51.54',
 '-0.17_51.55',
 '-0.16_51.46',
 '-0.16_51.47',
 '-0.16_51.48',
 '-0.16_

Nun erstellen des Grids

In [16]:
def fill_nan_with_neighbors(grid, radius=0.01, max_iterations=3, default_value=5.5):
    """
    Fills NaN values in the 'mean_trafficIndex' column iteratively with the mean of the neighbors
    within the radius. Remaining NaN values after max_iterations are assigned a default value.
    
    Parameters:
    - grid (pd.DataFrame): DataFrame containing the grid data.
    - radius (float): The radius to consider for the neighbors.
    - max_iterations (int): Maximum number of iterations to try filling NaN values.
    - default_value (float): Value to assign to remaining NaN values after max_iterations.
    
    Returns:
    - pd.DataFrame: DataFrame with NaN values filled.
    """
    df = grid.copy()
    
    def get_neighbors_mean(lat, long):
        """
        Returns the mean of the neighbors within the radius.
        """
        neighbors = df[
            (df['lat_rounded'] >= lat - radius) & (df['lat_rounded'] <= lat + radius) &
            (df['long_rounded'] >= long - radius) & (df['long_rounded'] <= long + radius)
        ]
        valid_neighbors = neighbors['mean_trafficIndex'].dropna()
        return valid_neighbors.mean() if not valid_neighbors.empty else np.nan

    for iteration in range(max_iterations):
        # Identify rows with NaN in 'mean_trafficIndex'
        nan_rows = df['mean_trafficIndex'].isna()
        if not nan_rows.any():
            # If there are no NaN values left, exit the loop
            break
        
        # Update NaN values with the mean of their neighbors
        df.loc[nan_rows, 'mean_trafficIndex'] = df[nan_rows].apply(
            lambda row: get_neighbors_mean(row['lat_rounded'], row['long_rounded']), axis=1
        )
    
    # Assign default value to any remaining NaN values
    df['mean_trafficIndex'].fillna(default_value, inplace=True)
    return df


In [ ]:
def advanced_grid(df, sensorid_col, trafficIndex_col, shape=0.01):
    """
    Create a grid with the mean trafficIndex for each grid cell. The grid is filled with all possible grid cells
    based on the minimum and maximum longitude and latitude in the input DataFrame.
    Input:
    - df: DataFrame containing sensor data with longitude and latitude
    - sensorid_col: column name for sensor ids
    - trafficIndex_col: column name for traffic indices (e.g. length or traffic volume)
    - shape: the size of the grid (diameter of the cell)
    
    Output:
    - A DataFrame with the grid and the mean trafficIndex for each grid cell.
    """
    
    decimal_places = abs(int(round(-np.log10(shape), 0)))

    df['long_rounded'] = df['long_rounded'].round(decimal_places)
    df['lat_rounded'] = df['lat_rounded'].round(decimal_places)
    
    df['grid_id'] = df['long_rounded'].astype(str) + "_" + df['lat_rounded'].astype(str)
    
    grid = df.groupby('grid_id').agg(
        mean_trafficIndex=(trafficIndex_col, 'mean'),
        sensors_in_grid=(sensorid_col, 'count'),
        long_rounded=('long_rounded', 'first'),
        lat_rounded=('lat_rounded', 'first')
    ).reset_index()
    
    complete_grid = create_complete_grid(df, shape)
    
    grid_filled = complete_grid.merge(grid[['grid_id', 'mean_trafficIndex', 'sensors_in_grid']], on='grid_id', how='left')
    grid_filled['mean_trafficIndex'] = grid_filled['mean_trafficIndex_y']
    grid_filled.drop(columns=['mean_trafficIndex_y', 'mean_trafficIndex_x'], inplace=True)
    
    grid_complete = fill_nan_with_neighbors(grid_filled)
    
    
       
    return grid_complete

In [24]:
full_grid = advanced_grid(df_real, sensorid_col='detid', trafficIndex_col='traffic', shape=0.01)

full_grid

(26,)
(10,)
     long_rounded  lat_rounded
0           -0.22        51.46
1           -0.22        51.47
2           -0.22        51.48
3           -0.22        51.49
4           -0.22        51.50
..            ...          ...
255          0.03        51.51
256          0.03        51.52
257          0.03        51.53
258          0.03        51.54
259          0.03        51.55

[260 rows x 2 columns]
     long_rounded  lat_rounded      grid_id
0           -0.22        51.46  -0.22_51.46
1           -0.22        51.47  -0.22_51.47
2           -0.22        51.48  -0.22_51.48
3           -0.22        51.49  -0.22_51.49
4           -0.22        51.50   -0.22_51.5
..            ...          ...          ...
255          0.03        51.51   0.03_51.51
256          0.03        51.52   0.03_51.52
257          0.03        51.53   0.03_51.53
258          0.03        51.54   0.03_51.54
259          0.03        51.55   0.03_51.55

[260 rows x 3 columns]


,long_rounded,lat_rounded,grid_id,sensors_in_grid,mean_trafficIndex
0,-0.22,51.46,-0.22_51.46,4.0,3.750000
1,-0.22,51.47,-0.22_51.47,3.0,2.466667
2,-0.22,51.48,-0.22_51.48,NaN,6.343850
3,-0.22,51.49,-0.22_51.49,11.0,6.196970
4,-0.22,51.50,-0.22_51.5,10.0,8.600000
...,...,...,...,...,...
255,0.03,51.51,0.03_51.51,22.0,1.168615
256,0.03,51.52,0.03_51.52,5.0,5.780000
257,0.03,51.53,0.03_51.53,1.0,0.714286
258,0.03,51.54,0.03_51.54,4.0,7.308333


In [25]:
test = full_grid.get('mean_trafficIndex').unique()
test

array([ 3.75      ,  2.46666667,  6.34385027,  6.1969697 ,  8.6       ,
        5.        ,  4.53333333,  6.66691919,  9.41666667, 11.70833333,
        2.88571429,  5.07677025,  8.        ,  8.71176471,  5.75      ,
        1.56666667,  5.17575758,  7.47501918, 10.29166667,  6.25      ,
        3.16666667,  4.82202381, 10.44632035,  8.12688172,  7.21923077,
        3.2989418 ,  3.730839  , 15.08888889,  6.02962963,  1.16666667,
        3.95619048,  4.61111111, 12.89738095,  4.19305556,  8.54166667,
        4.94749695, 14.16666667,  3.49350649,  4.98      ,  4.17619048,
        4.18154762,  3.57222222,  5.47727273,  3.88879552,  3.29166667,
        5.00740741,  7.2459707 ,  4.12380952,  5.48690476,  3.08861905,
        5.47321429,  6.37563025,  7.02222222, 13.62606516, 17.78720238,
       17.03528493,  5.04614122,  2.88      ,  0.        ,  0.8       ,
        3.11831502,  4.17142857,  4.97380952,  3.77447619, 17.21596639,
       12.09041353,  5.59791667,  1.48      ,  3.47585921,  3.42

In [27]:
def create_polygon(lat, long, shape='circle', size=0.005):
    """
    Create a polygon with different shapes (rectangle, octagon, triangle) around a central point.
    
    Args:
    - lat: Latitude of the center
    - long: Longitude of the center
    - shape: 'circle', 'rectangle', 'octagon', 'triangle'
    - size: the size of the shape (for polygons, it determines the distance of vertices from the center)
    
    Returns:
    - A list of [lat, long] tuples representing the vertices of the polygon.
    """
    
    if shape == 'rectangle':
        # Return a square (approximate rectangle) around the center
        return [
            [lat - size, long - size],  # bottom-left
            [lat - size, long + size],  # bottom-right
            [lat + size, long + size],  # top-right
            [lat + size, long - size]   # top-left
        ]
    
    elif shape == 'triangle':
        # Return an equilateral triangle (upward facing)
        return [
            [lat + size, long],              # top
            [lat - size / 2, long - size],   # bottom-left
            [lat - size / 2, long + size]    # bottom-right
        ]
    
    elif shape == 'octagon':
        # Create an approximate octagon (8-sided polygon) around the center
        angle_offset = np.pi / 4  # 45 degrees per side
        return [
            [lat + size * np.cos(i * angle_offset), long + size * np.sin(i * angle_offset)]
            for i in range(8)
        ]
    
    else:
        # Default to a circle (using folium.Circle)
        return None  # No polygon, as Circle will be used in the main function

def plot_grid_with_shapes(grid, shape='circle', city_center=(51.5074, -0.1278), zoom_start=12):
    """
    Plot the grid over a map of London with various shapes (circle, rectangle, octagon, triangle).
    - Red indicates higher mean traffic index.
    - Green indicates lower mean traffic index.
    
    Args:
    - grid: DataFrame containing grid information with mean traffic index, rounded lat/long, and grid_id.
    - shape: Shape to use for plotting ('circle', 'rectangle', 'octagon', 'triangle')
    - city_center: Tuple of (latitude, longitude) for the center of the map (default is central London).
    - zoom_start: Initial zoom level for the map (default is 12).
    
    Output:
    - Folium map with grid visualized.
    """
    # Create a Folium map centered around London
    m = folium.Map(location=city_center, zoom_start=zoom_start)

    # Create a color map that interpolates between green (low) and red (high)
    colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], 
                                 vmin=grid['mean_trafficIndex'].min(), 
                                 vmax=grid['mean_trafficIndex'].max())
    
    colormap.caption = 'Mean Traffic Index'
    m.add_child(colormap)  # Add the colormap to the map


    # Plot the grid cells on the map with the chosen shape
    for _, row in grid.iterrows():
        if row['mean_trafficIndex'] == "nan":
            continue
        
        color = colormap(row['mean_trafficIndex'])
        
        # Determine the vertices for the given shape
        polygon = create_polygon(row['lat_rounded'], row['long_rounded'], shape=shape)
        
        
        if shape == 'circle':
            # If shape is 'circle', use folium.Circle
            folium.Circle(
                location=[row['lat_rounded'], row['long_rounded']],
                radius=500,  # 500 meters radius (adjustable)
                color=color,
                fill=True,
                fill_opacity=0.6,
                popup=f"Grid ID: {row['grid_id']}<br>Mean Traffic Index: {row['mean_trafficIndex']}<br>Sensors in Grid: {row['sensors_in_grid']}"
            ).add_to(m)
        
        elif polygon:
            # If the shape is a polygon (rectangle, triangle, octagon), use folium.Polygon
            folium.Polygon(
                locations=polygon,
                color=color,
                fill=True,
                fill_opacity=0.6,
                popup=f"Grid ID: {row['grid_id']}<br>Mean Traffic Index: {row['mean_trafficIndex']}<br>Sensors in Grid: {row['sensors_in_grid']}"
            ).add_to(m)

    return m


In [33]:
map_with_rectangles1 = plot_grid_with_shapes(full_grid, shape='rectangle', city_center=(51.550, -0.021), zoom_start=15)

map_with_rectangles1

In [22]:
map_with_rectangles1 = plot_grid_with_shapes(grid_data, shape='rectangle', city_center=(51.550, -0.021), zoom_start=15)


map_with_rectangles1